In [1]:
#let's import the libraries
import numpy as np
import pandas as pd

In [2]:
books = pd.read_csv('/content/drive/MyDrive/bookfinal/book_final_data.csv',
                    usecols=['user_id', 'book_title', 'rating'],
                     dtype={'user_id': 'int32', 'book_title': 'str', 'rating': 'float32'})

In [3]:
books.head()

,user_id,rating,book_title
0,11676,8.0,Clara Callan
1,116866,9.0,Clara Callan
2,110912,10.0,Flu: The Story of the Great Influenza Pandemic...
3,197659,9.0,Flu: The Story of the Great Influenza Pandemic...
4,11676,9.0,The Kitchen God's Wife


In [4]:
combine_book_rating = books.dropna(axis = 0, subset = ['book_title'])
book_ratingCount = (combine_book_rating.
     groupby(by = ['book_title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['book_title', 'totalRatingCount']]
    )
book_ratingCount.head(10)

,book_title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,1
1,"Ask Lily (Young Women of Faith: Lily Series, ...",1
2,Dark Justice,1
3,Earth Prayers From around the World: 365 Pray...,4
4,Final Fantasy Anthology: Official Strategy Gu...,1
5,Flight of Fancy: American Heiresses (Zebra Ba...,1
6,Good Wives: Image and Reality in the Lives of...,2
7,Goosebumps Monster Edition 1: Welcome to Dead...,1
8,Highland Desire (Zebra Splendor Historical Ro...,1
9,Little Comic Shop of Horrors (Give Yourself G...,1


In [5]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'book_title', right_on = 'book_title', how = 'left')
rating_with_totalRatingCount.head()

,user_id,rating,book_title,totalRatingCount
0,11676,8.0,Clara Callan,2
1,116866,9.0,Clara Callan,2
2,110912,10.0,Flu: The Story of the Great Influenza Pandemic...,2
3,197659,9.0,Flu: The Story of the Great Influenza Pandemic...,2
4,11676,9.0,The Kitchen God's Wife,30


In [6]:
print(book_ratingCount['totalRatingCount'].describe())


count    79583.000000
mean         1.927899
std          3.438861
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max        137.000000
Name: totalRatingCount, dtype: float64


In [7]:
popularity_threshold=50
rating_popular_book= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.head()

,user_id,rating,book_title,totalRatingCount
11,7346,9.0,The Testament,61
12,11676,9.0,The Testament,61
13,13552,8.0,The Testament,61
14,30533,6.0,The Testament,61
15,31315,10.0,The Testament,61


In [8]:
rating_popular_book.shape


(4463, 4)

In [9]:
## First lets create a Pivot matrix

book_features_df=rating_popular_book.pivot_table(index='book_title',columns='user_id',values='rating').fillna(0)
book_features_df.head()

user_id,254,638,1424,1733,2033,2110,2276,2766,2891,2977,3363,3757,3827,4017,4385,5582,6073,6242,6251,6323,6543,6563,6575,7125,7158,7286,7346,7915,8066,8067,8245,8253,8454,8681,9417,9613,9856,10314,10354,10447,...,264321,264525,264657,265115,265313,265889,266056,266226,266865,267033,267635,267830,268030,268032,268110,268300,268657,268932,269566,269890,270352,270605,270713,270820,271195,271448,271622,271705,272225,273113,273718,273979,274004,274061,274301,275970,276680,277427,278137,278188
book_title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,9.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Painted House,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Time to Kill,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Angels & Demons,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0


In [10]:
from scipy.sparse import csr_matrix
book_features_df_matrix=csr_matrix(book_features_df)
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(book_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [11]:
book_features_df.shape


(66, 960)

In [12]:
query_index=np.random.choice(book_features_df.shape[0])
print(query_index)

31


In [13]:
distances, indices = model_knn.kneighbors(book_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)


In [14]:
for i in range (0,len(distances.flatten())):
  if i==0:
      print('Recommendations for {0}:\n'.format(book_features_df.index[query_index]))
  else:
     print('{0}: {1}, with distance of {2}:'.format(i, book_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Silence of the Lambs:

1: Red Dragon, with distance of 0.6763477325439453:
2: The Vampire Lestat (Vampire Chronicles, Book II), with distance of 0.6938213109970093:
3: Hannibal, with distance of 0.7082843780517578:
4: Jurassic Park, with distance of 0.7965760827064514:
5: A Time to Kill, with distance of 0.8047325015068054:
